In [1]:
CREATE DATABASE connector_test_db

Commands completed successfully.

Total execution time: 00:00:00.640

In [2]:
Use connector_test_db;
CREATE LOGIN connector_user1  WITH PASSWORD ='password123!#' 
CREATE USER connector_user1 FROM LOGIN connector_user1

-- to view data pool node configuration
grant VIEW DATABASE STATE to connector_user1

-- To create external tables in data pools
grant alter any external data source to connector_user1;

-- To create external table
grant create table to connector_user1;
grant alter any schema to connector_user1;



ALTER ROLE [db_datareader] ADD MEMBER connector_user1
ALTER ROLE [db_datawriter] ADD MEMBER connector_user1

Commands completed successfully.

Total execution time: 00:00:00.108

### **1\. Tables with computed columns**

In [1]:
-- append with computed columns at start and end
USE connector_test_db;

CREATE TABLE test_computed_col_1
(
    Date As (Year + '-' + Month),
    Id bigint,
    Year nvarchar(4),
    Month nvarchar(2),
    Years AS REPLICATE(Year, 2)
)

Commands completed successfully.

Total execution time: 00:00:00.011

In [2]:
-- append with 1 computed column in between
USE connector_test_db;
CREATE TABLE test_computed_col_2
(
    Id bigint,
    Date As (Year + '-' + Month),
    Year nvarchar(4),
    Month nvarchar(2)
)

Commands completed successfully.

Total execution time: 00:00:00.010

In [3]:
-- append with 2 computed columns in between
USE connector_test_db;

CREATE TABLE test_computed_col_3
(
    Id bigint,
    Date As (Year + '-' + Month),
    Year nvarchar(4),
    Years AS REPLICATE(Year, 2),
    Month nvarchar(2)
)

Commands completed successfully.

Total execution time: 00:00:00.011

In [4]:
-- append with 1 computed column in df and table, but table has 1 less col
USE connector_test_db;

CREATE TABLE test_computed_col_4
(
    Id bigint,
    Year nvarchar(4),
    Years AS REPLICATE(Year, 2)
)

Commands completed successfully.

Total execution time: 00:00:00.012

### **2\. Temporal Tables (Generated Always)**

In [13]:
USE connector_test_db;
CREATE TABLE dbo.tempTest
(
  Id bigint NOT NULL PRIMARY KEY CLUSTERED
  , Year nvarchar(4)
  , Month nvarchar(2)
  , [ValidFrom] datetime2 GENERATED ALWAYS AS ROW START
  , [ValidTo] datetime2 GENERATED ALWAYS AS ROW END
  , PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo)
 )
WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.tempTestHistory));

Commands completed successfully.

Total execution time: 00:00:00.015

In [27]:
-- create table with comptued cols and then convert it to temporal tables
USE connector_test_db;
CREATE TABLE test_computed_temp
(
    Date As (Year + '-' + Month),
    Id bigint PRIMARY KEY NOT NULL,
    Year nvarchar(4),
    Month nvarchar(2),
    Years AS REPLICATE(Year, 2)
)

-- Convert test_computed_col_1 to temporal tables
ALTER TABLE test_computed_temp
    ADD 
        [ValidFrom] datetime2 GENERATED ALWAYS AS ROW START DEFAULT GETUTCDATE()
        , [ValidTo] datetime2 GENERATED ALWAYS AS ROW END DEFAULT CONVERT(DATETIME2, '9999-12-31 23:59:59.9999999')
        , PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo)

ALTER TABLE test_computed_temp SET (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.test_computed_temp_history));

Commands completed successfully.

Total execution time: 00:00:00.032

In [ ]:
-- Use below to drop temporal tables
/*
USE connector_test_db;
ALTER TABLE dbo.tempTest SET (SYSTEM_VERSIONING = OFF);
DROP TABLE dbo.tempTest;
DROP TABLE dbo.tempTestHistory;
*/

/*
USE connector_test_db;
ALTER TABLE dbo.test_computed_temp SET (SYSTEM_VERSIONING = OFF);
DROP TABLE dbo.test_computed_temp;
DROP TABLE test_computed_temp_history;
*/

### **3\. Graph Table**

In [11]:
USE connector_test_db
GO

CREATE TABLE [dbo].[ForumPosts](
    [PostID] [bigint] NULL,
    [PostTitle] [varchar](100) NULL,
    [PostBody] [varchar](1000) NULL
) AS NODE
GO

CREATE TABLE [dbo].[Replies]
AS EDGE
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.027